# แบบฝึกหัด 2

---

จากข้อมูล titanic.csv ให้ทำการวิเคราะห์ข้อมูลเบื้องต้นเกี่ยวกับคอลัมน์ cabin  
โดยแผนภาพด้านล่างนี้เป็นตำแหน่งของ cabin ตามตัวอักษรตัวแรกของรหัส cabin ที่บันทึกในชุดข้อมูล

![titanic-cabin](https://upload.wikimedia.org/wikipedia/commons/thumb/0/0d/Olympic_%26_Titanic_cutaway_diagram.png/800px-Olympic_%26_Titanic_cutaway_diagram.png)

ให้นิสิตสร้างคอลัมน์เพิ่ม โดยคอลัมน์ที่เพิ่มมานี้ จะเก็บข้อมูลของตัวอักษรตัวแรกที่ปรากฏในคอลัมน์ Cabin เช่น  
- หากในคอลัมน์ Cabin เป็นค่า "C123" ในคอลัมน์ที่สร้างจะบันทึก "C"  
- หากในคอลัมน์ Cabin เป็นค่า "C23 C25 C27" ในคอลัมน์ที่สร้างจะบันทึก "C"
- หากในคอลัมน์ Cabin เป็นค่า "B58 B60 C2" ในคอลัมน์ที่สร้างจะบันทึก "B"
- หากในคอลัมน์ Cabin เป็นค่า "F2" ในคอลัมน์ที่สร้างจะบันทึก "F"
- หากในคอลัมน์ Cabin เป็นค่า NaN ในคอลัมน์ที่สร้างจะบันทึก NaN

จากนั้นให้นิสิตเขียนโค้ดเพื่อตอบคำถามในแต่ละข้อ

In [113]:
import numpy as np
import pandas as pd

In [114]:
df = pd.read_csv("https://raw.githubusercontent.com/ThammakornS/ProgStat/main/dataset/titanic.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


นิสิตแสดงวิธีสร้างคอลัมน์

In [115]:
df['cabin_letter'] = df.Cabin.str.get(0)
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,cabin_letter
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,NaN
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,NaN
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,C
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,NaN


1. ในแต่ละตัวอักษร Cabin มีจำนวนผู้โดยสารกี่คน และมีจำนวน NaN เท่าไหร่

In [116]:
df.cabin_letter.value_counts()
df.cabin_letter.value_counts(dropna=False)


cabin_letter
NaN    687
C       59
B       47
D       33
E       32
A       15
F       13
G        4
T        1
Name: count, dtype: int64

2. ในแต่ละตัวอักษร Cabin มีจำนวนผู้โดยสารที่รอดและเสียชีวิตอย่างละเท่าไหร่ cabin ใดมีจำนวนผู้เสียชีวิตมากที่สุด

In [117]:
print(pd.crosstab(df.cabin_letter,df.Survived,margins=False))
dead = df[df['Survived'] == 0]
dead_counts = dead['cabin_letter'].value_counts()
print(f"{dead_counts.head(1)}")

Survived       0   1
cabin_letter        
A              8   7
B             12  35
C             24  35
D              8  25
E              8  24
F              5   8
G              2   2
T              1   0
cabin_letter
C    24
Name: count, dtype: int64


3. cabin ใดมีค่า สัดส่วน จำนวนผู้รอดชีวิตหารด้วยจำนวนผู้เสียชีวิต สูงสุด

In [118]:
sur = pd.crosstab(df['cabin_letter'], df['Survived'])
sur['rate'] = sur[1]/sur[0]
surrate = sur.sort_values('rate',ascending=False)
print("ในกรณีที่ไม่เอา NaN มาคำนวณ")
print(f"{surrate.head(1)}")



ในกรณีที่ไม่เอา NaN มาคำนวณ
Survived      0   1   rate
cabin_letter              
D             8  25  3.125


4. cabin ใดมีค่าเฉลี่ยราคาตั๋ว (Fare) สูงสุด

In [119]:
cabin_favg = df.groupby('cabin_letter')['Fare'].mean()
cabin_favgsort = cabin_favg.sort_values(ascending=False)
print(f"cabin ที่มีค่าเฉลี่ยราคาตั๋ว (Fare) สูงสุด")
print(f"{cabin_favgsort.head(1)}")

cabin ที่มีค่าเฉลี่ยราคาตั๋ว (Fare) สูงสุด
cabin_letter
B    113.505764
Name: Fare, dtype: float64


5. จงแสดงค่าเฉลี่ยราคาตั๋ว (Fare) ของผู้รอดชีวิต และผู้เสียชีวิต ในแต่ละ cabin หากเปรียบเทียบราคาเฉลี่ยของผู้ที่รอดและเสียชีวิตจะพบว่า

In [120]:
cabin_favg1 = df.groupby(['cabin_letter' , 'Survived'])['Fare'].mean().unstack()
cabin_favg1['fare_dif'] = cabin_favg1[1] - cabin_favg1[0]
print('จะเห็นได้ว่าค่าเฉลี่ยราคาตั๋วของผู้รอดชีวิตจะมีราคาที่มากกว่า')
cabin_favg1

จะเห็นได้ว่าค่าเฉลี่ยราคาตั๋วของผู้รอดชีวิตจะมีราคาที่มากกว่า


Survived,0,1,fare_dif
cabin_letter,,,
A,35.496350,44.341071,8.844721
B,57.137500,132.832026,75.694526
C,99.972396,100.274046,0.301650
D,53.770825,58.356176,4.585351
E,43.715625,46.797050,3.081425
F,11.340000,23.294787,11.954787
G,10.462500,16.700000,6.237500
T,35.500000,NaN,NaN


6. ในแต่ละอักษร cabin มีจำนวนผู้โดยสารแยกตาม Pclass อย่างละกี่คน

In [121]:
sur = pd.crosstab(df['cabin_letter'], df['Pclass'])
sur

Pclass,1,2,3
cabin_letter,,,
A,15,0,0
B,47,0,0
C,59,0,0
D,29,4,0
E,25,4,3
F,0,8,5
G,0,0,4
T,1,0,0
